In [1]:
import numpy as np
from models.base_models import Transformer
from arithmetic_sampler import ArithmeticSampler
from config import get_config
from train import train
import torch
import torch.nn as nn
import torch.nn.functional as F
import utils as u

%load_ext autoreload
%autoreload 2

In [20]:
config = get_config()
sampler = ArithmeticSampler(config.task.max_variables)
torch.cuda.empty_cache()

In [16]:
%%timeit
sampler.generate(2**17)

2.39 s ± 49.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
model = Transformer(config)

train(model, sampler, config, verbose=False)

Results are saved in:  results\train_d24c5466a5868d3ea5c32f25b224f4b4


Button(description='Stop Training', style=ButtonStyle())

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Transformer                              [32, 82]                  [32, 82, 16]              --
├─Embedding: 1-1                         [32, 82]                  [32, 82, 64]              1,024
├─ModuleList: 1-2                        --                        --                        --
│    └─TFBlock: 2-1                      [32, 82, 64]              [32, 82, 64]              --
│    │    └─LayerNorm: 3-1               [32, 82, 64]              [32, 82, 64]              128
│    │    └─MultiHeadAttention: 3-2      [32, 82, 64]              [32, 82, 64]              16,384
│    │    └─Dropout: 3-3                 [32, 82, 64]              [32, 82, 64]              --
│    │    └─LayerNorm: 3-4               [32, 82, 64]              [32, 82, 64]              128
│    │    └─Sequential: 3-5              [32, 82, 64]              [32, 82, 64]              16,576
│    │    └─Dropout: 3

eval/accuracy,▁▃▅▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████████
eval/loss,█▅▅▄▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/op-accuracy,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▄▄▅▅▇▇▇▇▇██████████
train/accuracy,▁▁▂▅▅▇▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
train/loss,█▇▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▁▂▂▃▄▅▅▅▇▇▇██▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▂▂▁
train/op-accuracy,▁▁▁▁▂▁▂▂▂▁▁▂▁▂▁▂▂▂▂▁▂▄▄▅▄▆▇▇▆▇▇▇██▇█████
eval/accuracy,0.9967
eval/loss,0.01043
eval/op-accuracy,0.98877
train/accuracy,0.99848


Start training...


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

Training complete.


In [17]:
batch, mask = sampler.generate(1)
sampler.decode(batch)

['                                        (2+(6-(8-3)))-((8+1)+5)=(2+(6-5))-((8+1)+5)']

In [18]:
batch = batch.to(config.device)
logits, _ = model(batch)
(logits[:, :-1].argmax(dim=-1) - batch[:, 1:])[mask[:,1:]>0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

In [19]:
sampler.decode(logits[:, :-1].cpu().argmax(dim=-1)[mask[:,1:]>0])

['(2+(6-5))-((8+1)+5)']